This one is for testing some FCNNs

In [ ]:
import pandas as pd
import os, sys
import numpy as np
from typing import Tuple
import matplotlib.pyplot as plt
import torch

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import fcnn_models

In [ ]:
def load_csv_data(file_path: str) -> Tuple[np.ndarray, np.array, np.array]:
    """
    Reads the CSV with all the training data: grayscale log-mel spectrogram pixels, label and author of each sample
    And extracts them respectively, resizing the features to fit the CNN input shape (channel, height, width).

    Args:
        file_path (str): Path to the CSV file containing training data.

    Returns:
        Tuple[np.ndarray, np.array, np.array]: Returns features, labels, and authors from the CSV file. Features shape is (N x 70,112),
        while labels and authors are 1D arrays of size N, where N is the number of samples.
    """
    # Load training data
    df = pd.read_csv(file_path)

    print(f"Dataset shape: {df.shape}")
    print(f"Number of classes: {df['label'].nunique()}")
    print(f"Number of authors: {df['author'].nunique()}")

    # Extract labels, authors, and features
    labels = df['label'].values.astype(np.int64)
    authors = df['author'].values
    features = df.drop(columns=['label', 'author']).values.astype(np.float32)
    print(f"Features shape before reshape: {features.shape} (should be N x 70112!)")

    # Convert to 0-1 range
    features /= 255.0

    print("Features shape:", features.shape)
    print("Labels shape:", labels.shape)
    print("Authors shape:", authors.shape)
    print("Unique classes:", len(np.unique(labels)))
    print("Unique authors:", len(np.unique(authors)))

    # No need for df variable after extracting features, release memory
    del df

    return features, labels, authors

In [ ]:
# Keep the old CSV-based loading for compatibility (uncomment if needed)
train_data_path = os.path.join('..', 'database', 'meta', 'final', 'train_data.csv')
features, labels, authors = load_csv_data(train_data_path)

# Display class distribution
plt.figure(figsize=(12, 6))
unique_labels, counts = np.unique(labels, return_counts=True)
plt.bar(unique_labels, counts, alpha=0.7)
plt.xlabel('Class ID')
plt.ylabel('Number of Samples')
plt.title('Class Distribution in Training Data')
plt.xticks(unique_labels)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Average samples per class: {len(labels) / len(unique_labels):.1f}")

In [ ]:
# Split with a set seed, las que ya encontre arriba, para tardar menos
from utils.split import get_set_seed_indices, display_split_statistics
seed_single = 245323 # Quality: 0.2671

train_indices, val_indices, best_score, seed = get_set_seed_indices(
    features=features,
    labels=labels, 
    authors=authors,
    test_size=0.2,
    seed=seed_single)

# Extract train and validation data using indices
trFeatures = features[train_indices]
trLabels = labels[train_indices]
trAuthors = authors[train_indices]

vFeatures = features[val_indices]
vLabels = labels[val_indices]
vAuthors = authors[val_indices]

print(f"Training set: {trFeatures.shape[0]} samples")
print(f"Validation set: {vFeatures.shape[0]} samples")
print(f"Split quality score: {best_score}")

display_split_statistics((train_indices, val_indices, best_score, seed), "single")

# Convert labels to one-hot encoding for Nn model
num_classes = len(np.unique(labels))

def to_one_hot(labels, num_classes):
    """Convert integer labels to one-hot encoding"""
    one_hot = np.zeros((len(labels), num_classes))
    one_hot[np.arange(len(labels)), labels] = 1
    return one_hot

# One-hot encode labels for Nn model
trLabels_onehot = to_one_hot(trLabels, num_classes)
vLabels_onehot = to_one_hot(vLabels, num_classes)

print(f"Original labels shape: {trLabels.shape}")
print(f"One-hot labels shape: {trLabels_onehot.shape}")
print(f"Number of classes: {num_classes}")

In [ ]:
from utils.fcnn_models import ... # import the models here

# Models List
models = { ... } # fill the list (dictionary {'BirdFCNN_v3': BirdCNN_v3, etc.})